I assume that it's worth mentioning that it's not so easy make codespaces working on Ubuntu (and should be the same for all other linuxes).

In [1]:
import pandas as pd
# !pip install pyarrow
import seaborn as sns
import matplotlib.pyplot as plt
import pickle


In [2]:
from sklearn.feature_extraction  import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [3]:
categorical=['PULocationID','DOLocationID']
numerical=['trip_distance']

There isn't the link https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page on the github page

In [4]:
def read_dataframe(filename):
    categorical=['PULocationID','DOLocationID']
    df = pd.read_parquet(filename)
    print(filename,len(df.columns),df.columns,sep="\n")

    if 'lpep_dropoff_datetime' in df.columns:
        date_dropoff_col_name = 'lpep_dropoff_datetime'
    elif 'tpep_dropoff_datetime' in df.columns:
        date_dropoff_col_name = 'tpep_dropoff_datetime'
    else:
        print("No appropriate drop-off datetime column exists in the dataframe.")
        return None

    # Similar check for pickup datetime column
    if 'lpep_pickup_datetime' in df.columns:
        date_pickup_col_name = 'lpep_pickup_datetime'
    elif 'tpep_pickup_datetime' in df.columns:
        date_pickup_col_name = 'tpep_pickup_datetime'
    else:
        print("No appropriate pick-up datetime column exists in the dataframe.")
        return None

    df[date_dropoff_col_name] = pd.to_datetime(df[date_dropoff_col_name])
    df[date_pickup_col_name] = pd.to_datetime(df[date_pickup_col_name])
    
    df['duration'] = df[date_dropoff_col_name] - df[date_pickup_col_name]
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60 )
    # df=df[df.trip_type==2]
    # sns.displot(df.duration)
    # df= df[(df.duration >= 0)]
    print(df.duration.describe(percentiles=[x/10 for x in range(1,9)]+[x/100+.9 for x in range(10)]),df.duration.std())
    count=df.count()
    df= df[(df.duration >= 1) & (df.duration <=60 )]
    print(df.count()/count)

    # df.duration.describe(percentiles=[x/10 for x in range(1,9)]+[x/100+.9 for x in range(10)])
    df[categorical]=df[categorical].astype(str)
    return df


In [5]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_val =  read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
19
Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')
count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
10%      4.483333e+00
20%      6.283333e+00
30%      7.950000e+00
40%      9.650000e+00
50%      1.151667e+01
60%      1.373333e+01
70%      1.651667e+01
80%      2.053333e+01
90%      2.794167e+01
91%      2.915000e+01
92%      3.055000e+01
93%      3.216667e+01
94%      3.410000e+01
95%      3.646667e+01
96%      3.945000e+01
97%      4.335000e+01
98%      4.873333e+01
99%      5.725000e+01
max      1.002918e+04
Name: duration, dtype: floa

Q1: 19
Q2: 42.59
Q3: 98?

In [6]:
len(df_train),len(df_val)

(3009173, 2855951)

df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

categorical=['PU_DO']

In [13]:
dv=DictVectorizer()
train_dicts = df_train[categorical].to_dict(orient='records')
X_train=dv.fit_transform(train_dicts)
val_dicts = df_val[categorical].to_dict(orient='records')
X_val=dv.transform(val_dicts)


dv=DictVectorizer()
train_dicts = df_train[categorical+numerical].to_dict(orient='records')
X_train=dv.fit_transform(train_dicts)
val_dicts = df_val[categorical+numerical].to_dict(orient='records')
X_val=dv.transform(val_dicts)


In [14]:
X_val.shape

(2855951, 515)

Q4: 515

In [15]:
target='duration'
y_train=df_train[target].values
y_val=df_val[target].values

In [18]:
lr=LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train,y_pred,squared=False)

7.6492610279057605

Q5: 7.64

In [19]:
mean_squared_error(y_val,lr.predict(X_val),squared=False)

7.81183265470218

Q6: 7.81

lr=Lasso(alpha=0.0001)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val,y_pred,squared=False)

lr=Ridge(0.0001)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val,y_pred,squared=False)

In [12]:
# !mkdir models
# with open('models/lin_reg.bin', 'wb') as f_out:
#     pickle.dump((dv,lr), f_out)